In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from transformers import BertTokenizer, TFBertModel
import numpy as np

# Load the dataset
data = pd.read_csv("/content/drive/MyDrive/Reviews.csv")

# Display first few rows of the data
print(data.head())

# Convert Score into sentiment labels: 1-3 as Negative, 4-5 as Positive
data['Sentiment'] = data['Score'].apply(lambda x: 1 if x > 3 else 0)

# Split data into training and testing sets
X = data['Text']
y = data['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the text data for traditional ML models using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

   Id   ProductId          UserId                      ProfileName  \
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian   
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time  \
0                     1                       1      5  1303862400   
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [ ]:
# Logistic Regression Model
lr_model = LogisticRegression()
lr_model.fit(X_train_vec, y_train)
lr_pred = lr_model.predict(X_test_vec)
print("Logistic Regression Accuracy:", accuracy_score(y_test, lr_pred))
print(classification_report(y_test, lr_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.8972829863401677
              precision    recall  f1-score   support

           0       0.81      0.69      0.74     24666
           1       0.92      0.96      0.94     89025

    accuracy                           0.90    113691
   macro avg       0.86      0.82      0.84    113691
weighted avg       0.89      0.90      0.89    113691



In [ ]:
# Import the necessary libraries for Naive Bayes and LightGBM
from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier

# Naive Bayes Model
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)
nb_pred = nb_model.predict(X_test_vec)
print("Naive Bayes Accuracy:", accuracy_score(y_test, nb_pred))
print(classification_report(y_test, nb_pred))

# LightGBM Model
lgbm_model = LGBMClassifier()
lgbm_model.fit(X_train_vec, y_train)
lgbm_pred = lgbm_model.predict(X_test_vec)
print("LightGBM Accuracy:", accuracy_score(y_test, lgbm_pred))
print(classification_report(y_test, lgbm_pred))


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Naive Bayes Accuracy: 0.835633427448083
              precision    recall  f1-score   support

           0       0.89      0.28      0.42     24666
           1       0.83      0.99      0.90     89025

    accuracy                           0.84    113691
   macro avg       0.86      0.63      0.66    113691
weighted avg       0.84      0.84      0.80    113691

[LightGBM] [Info] Number of positive: 354752, number of negative: 100011
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 38.577013 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 733959
[LightGBM] [Info] Number of data points in the train set: 454763, number of used features: 5000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.780081 -> initscore=1.266139
[LightGBM] [Info] Start training from score 1.266139
LightGBM Accuracy: 0.8802279863841465
              precision    recall  f1-score   support

           0       0.82      0.57      0

In [ ]:
# Implement k-fold cross-validation
for model, name in zip([lr_model, nb_model , lgbm_model], ['Logistic Regression', 'MultinomialNB', 'LGBMClassifier']):
    scores = cross_val_score(model, X_train_vec, y_train, cv=5)
    print(f"{name} Cross-Validation Accuracy: {np.mean(scores)}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Logistic Regression Cross-Validation Accuracy: 0.8959589944237776
MultinomialNB Cross-Validation Accuracy: 0.8328294087641129
[LightGBM] [Info] Number of positive: 283802, number of negative: 80008
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 39.849087 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 733987
[LightGBM] [Info] Number of data points in the train set: 363810, number of used features: 5000
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.780083 -> initscore=1.266150
[LightGBM] [Info] Start training from score 1.266150
[LightGBM] [Info] Number of positive: 283801, number of negative: 80009
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 42.711156 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 733983
[LightGBM] [Info] Number of data points in the train set: 363810, number of used features: 5000
[LightGB

In [ ]:
# Deep Learning Model - LSTM

In [ ]:
# Tokenization and Padding
tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding the sequences
max_length = 100  # maximum length of sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

# Define the LSTM model
lstm_model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.5),
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the model
lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
lstm_model.fit(X_train_pad, y_train, epochs=2, batch_size=64, validation_data=(X_test_pad, y_test))

# Evaluate the model
lstm_pred = (lstm_model.predict(X_test_pad) > 0.5).astype("int32")
print("LSTM Accuracy:", accuracy_score(y_test, lstm_pred))
print(classification_report(y_test, lstm_pred))


Epoch 1/2
7106/7106 ━━━━━━━━━━━━━━━━━━━━ 3171s 445ms/step - accuracy: 0.8819 - loss: 0.2884 - val_accuracy: 0.9214 - val_loss: 0.1944
Epoch 2/2
7106/7106 ━━━━━━━━━━━━━━━━━━━━ 3207s 446ms/step - accuracy: 0.9321 - loss: 0.1758 - val_accuracy: 0.9346 - val_loss: 0.1698
3553/3553 ━━━━━━━━━━━━━━━━━━━━ 212s 60ms/step
LSTM Accuracy: 0.9345946468937735
              precision    recall  f1-score   support

           0       0.86      0.83      0.85     24666
           1       0.95      0.96      0.96     89025

    accuracy                           0.93    113691
   macro avg       0.91      0.90      0.90    113691
weighted avg       0.93      0.93      0.93    113691

